In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import fbeta_score, make_scorer, confusion_matrix, cohen_kappa_score, precision_score, recall_score, classification_report, f1_score
from sklearn.preprocessing import OneHotEncoder,  StandardScaler
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier


from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor, AdaBoostClassifier

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error

In [ ]:
heart_data = "../data/clean/resampled_data.csv"
heart_df = pd.read_csv(heart_data)
heart_df.head()

In [ ]:
heart_df.shape

In [ ]:
df_new = heart_df.drop(['PhysHlth', 'DiffWalk','Education'],  axis=1)
df_new

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(data=heart_df, x='HeartDiseaseorAttack')
plt.title(f'Count plot')
plt.xticks(rotation=45)  # Rotate x-axis labels if needed
plt.show()

In [ ]:
target = df_new['HeartDiseaseorAttack']
features = df_new.drop('HeartDiseaseorAttack', axis=1)

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

#Normalise all columns to be 0-1
normalizer = MinMaxScaler()
normalizer.fit(x_train)

x_train_norm = normalizer.transform(x_train)
x_test_norm = normalizer.transform(x_test)

x_train_norm = pd.DataFrame(x_train_norm, columns=x_train.columns, index=x_train.index )
x_test_norm = pd.DataFrame(x_test_norm, columns=x_test.columns, index=x_test.index)

#### Adaptive boosting

#### Using classifier

In [ ]:
ada_reg = AdaBoostClassifier(DecisionTreeClassifier(max_depth=20),
                            n_estimators=100)
ada_reg.fit(x_train_norm, y_train)
pred = ada_reg.predict(x_test_norm)
print(f"MAE, {mean_absolute_error(pred, y_test): .2f}")
print(f"MSE, {mean_squared_error(pred, y_test): .2f}")
print(f"RMSE, {root_mean_squared_error(pred, y_test): .2f}")
print(f"R2 score, {ada_reg.score(x_test_norm, y_test): .2f}")

#### Interpretation of the result

The MAE, MSE, and RMSE values suggest relatively small prediction errors, indicating that the model's predictions are quite close to the actual values.

The R² score of 0.76 means that the model is able to explain 76% of the variation in the target variable, which indicates that it's doing a good job of predicting the outcomes.

In [ ]:
# Iterate over a range of n_estimators for AdaBoost
for i in range(3, 8): 
    ada_boost = AdaBoostClassifier(DecisionTreeClassifier(max_depth=20),
        n_estimators=i,  # number of estimators (trees)
        random_state=42
    )
    print(f"For {i} n_estimators:")
    # Fit the AdaBoost model
    ada_boost.fit(x_train, y_train)
    # Predict the labels on the test set
    y_pred = ada_boost.predict(x_test)
    
    # Calculate Cohen's Kappa score
    kappa = cohen_kappa_score(y_test, y_pred)
    
    # Calculate accuracy manually using confusion matrix
    accuracy = 100 * ((cm[0][0] + cm[1][1]) / (sum(cm[0]) + sum(cm[1])))
    
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"Cohen's Kappa: {kappa:.4f}")
    print(cm)